In [1]:
import math
import logging
import numpy as np
import pandas as pd
import scipy.signal as sig
import altair as alt

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.load import graph_melt, load_normalize
from spinorama.load_parse import parse_graphs_speaker
from spinorama.load_rewseq import parse_eq_iir_rews
from spinorama.graph import graph_spinorama, graph_freq
from spinorama.compute_normalize import normalize_mean, normalize_cea2034
from spinorama.compute_scores import speaker_pref_rating
from spinorama.compute_cea2034 import compute_cea2034, estimated_inroom_HV, spl2pressure, pressure2spl
from spinorama.filter_iir import Biquad

In [2]:
# df_speaker = parse_graphs_speaker('../datas/', 'Genelec', 'Genelec 8341A')
# df_speaker = parse_graphs_speaker('../datas/', 'Focal', 'Focal Aria 906')
# df_speaker = parse_graphs_speaker('../datas/', 'Elac', 'Elac Debut 2.0 B6.2')
df_speaker = parse_graphs_speaker('../datas/', 'Adam', 'Adam S2V', 'klippel', 'asr')

spin  = df_speaker['CEA2034_unmelted']
pir   = estimated_inroom_HV(df_speaker['SPL Horizontal_unmelted'], df_speaker['SPL Vertical_unmelted'])
score = speaker_pref_rating(graph_melt(spin), graph_melt(pir))



In [39]:
def parametric_eq_build(freq, peq):
    filter = 0
    for w, iir in peq:
        filter += w*np.array([iir.log_result(f) for f in freq])
    return filter


def parametric_eq_apply(spl, peq):
    filter = 0
    for w, iir in peq:
        filter += w*np.array([iir(v) for v in spl])
    return filter


def iir_graph(spin, measurement, peq):
    spin_freq   = spin['Freq'].to_numpy()
    mean = np.mean(spin.loc[(spin.Freq>500) & (spin.Freq<10000)]['On Axis'])
    curve = spin[measurement]-mean
    filter = parametric_eq_build(spin_freq, peq)
    curve_filtered = parametric_eq_apply(curve, peq)
    dff = pd.DataFrame({'Freq': spin_freq, measurement: curve, '{0} Filtered'.format(measurement): curve_filtered, 'Filter': filter})
    return alt.Chart(graph_melt(dff)).mark_line(clip=True).encode(
        x=alt.X('Freq:Q', scale=alt.Scale(type='log', nice=False, domain=[20, 20000])),
        y=alt.Y('dB:Q', scale=alt.Scale(domain=[-25, 5])),
        color=alt.Color('Measurements')
    )

def iir_apply(spl, peq):
    freq   = spl['Freq'].to_numpy()
    mean = np.mean(spl.loc[(spin.Freq>500) & (spl.Freq<10000)]['On Axis'])
    ddf = []
    ddf.append(pd.DataFrame({'Freq': freq}))
    for angle in spl.keys():
        if angle == 'Freq':
            continue
        curve = spl[angle]-mean
        # curve_filtered = curve+parametric_eq_apply(curve, peq)
        curve_filtered = curve + parametric_eq_build(freq, peq)
        #print('{0:7s} range [{1:.1f}, {2:.1f}] filtered [{3:.1f}, {4:.1f}]'.format(angle, np.min(curve), np.max(curve), np.min(curve_filtered), np.max(curve_filtered)))
        ddf.append(pd.DataFrame({angle: curve_filtered}))
    f = pd.concat(ddf, axis=1)
    return f.dropna()



In [40]:
my_fs = 48000
my_peq = [
    (0, Biquad(Biquad.HIGHPASS, 50, my_fs, 0.2, 1)),
    (1, Biquad(Biquad.PEAK, 715, my_fs, 8, -2.8)),
    (0, Biquad(Biquad.NOTCH, 1400, my_fs, 0.1, 0.1)),
]

my_peq = parse_eq_iir_rews('../datas/eq/Adam S2V/iir.txt', my_fs)
ddf_horizontal = iir_apply(df_speaker['SPL Horizontal_unmelted'], my_peq)
ddf_vertical   = iir_apply(df_speaker['SPL Vertical_unmelted'], my_peq)

spin_filtered = graph_melt(compute_cea2034(ddf_horizontal, ddf_vertical))
pir_filtered  = graph_melt(estimated_inroom_HV(ddf_horizontal, ddf_vertical))
score_filtered = speaker_pref_rating(spin_filtered, pir_filtered)

In [41]:
for i in range(0, len(my_peq)):
    if my_peq[i][0] != 0:
        print(my_peq[i][1])
print('         SPK FLT')
print('-----------------')
print('NBD  ON {0:0.2f} {1:0.2f}'.format(score['nbd_on_axis'], score_filtered['nbd_on_axis']))
print('NBD  LW {0:0.2f} {1:0.2f}'.format(score['nbd_listening_window'], score_filtered['nbd_listening_window']))
print('NBD PIR {0:0.2f} {1:0.2f}'.format(score['nbd_pred_in_room'], score_filtered['nbd_pred_in_room']))
print('SM  PIR {0:0.2f} {1:0.2f}'.format(score['sm_pred_in_room'], score_filtered['sm_pred_in_room']))
print('SM   SP {0:0.2f} {1:0.2f}'.format(score['sm_sound_power'], score_filtered['sm_sound_power']))
print('LFX       {0}   {1}'.format(score['lfx_hz'], score_filtered['lfx_hz']))
print('LFQ     {0} {1}'.format(score['lfq'], score_filtered['lfq']))
print('-----------------')
print('Score    {0}  {1}'.format(score['pref_score'], score_filtered['pref_score']))
print('-----------------')

Type:3,Freq:340.0,Rate:48000.0,Q:10.0,Gain:0.8
Type:3,Freq:400.0,Rate:48000.0,Q:15.0,Gain:0.7
Type:3,Freq:485.0,Rate:48000.0,Q:10.0,Gain:1.5
Type:3,Freq:597.0,Rate:48000.0,Q:5.0,Gain:-1.5
Type:3,Freq:1049.0,Rate:48000.0,Q:1.0,Gain:-1.8
Type:3,Freq:1370.0,Rate:48000.0,Q:20.0,Gain:-0.5
Type:3,Freq:1660.0,Rate:48000.0,Q:20.0,Gain:0.8
Type:3,Freq:1940.0,Rate:48000.0,Q:40.0,Gain:0.8
Type:3,Freq:2350.0,Rate:48000.0,Q:20.0,Gain:-1.2
Type:3,Freq:2902.0,Rate:48000.0,Q:4.2,Gain:-1.1
Type:3,Freq:3979.0,Rate:48000.0,Q:5.0,Gain:1.4
Type:3,Freq:5886.0,Rate:48000.0,Q:5.0,Gain:3.1
Type:3,Freq:6702.0,Rate:48000.0,Q:1.0,Gain:-3.8
Type:3,Freq:7943.0,Rate:48000.0,Q:5.0,Gain:3.1
Type:3,Freq:8825.0,Rate:48000.0,Q:5.0,Gain:-2.7
Type:3,Freq:9458.0,Rate:48000.0,Q:5.0,Gain:2.8
Type:3,Freq:10680.0,Rate:48000.0,Q:10.0,Gain:-0.7
Type:3,Freq:11866.0,Rate:48000.0,Q:1.0,Gain:-1.8
Type:3,Freq:11950.0,Rate:48000.0,Q:12.0,Gain:0.8
Type:3,Freq:17828.0,Rate:48000.0,Q:5.0,Gain:-1.6
         SPK FLT
-----------------
NBD  O

In [49]:
def compute_differences(df1, df2):
    df = df1.copy().reset_index()
    dfc = df2.copy().reset_index()
    df['dB'] -= dfc['dB']
    return df.dropna()

In [50]:
g_spin = df_speaker['CEA2034']
delta = compute_differences(spin_filtered, g_spin)

In [51]:
g_params = {'xmin': 20, 'xmax': 20000, 'ymin': -40, 'ymax': 10, 'width': 400, 'height': 250}
g_delta  = {'xmin': 20, 'xmax': 20000, 'ymin': -25, 'ymax': 25, 'width': 400, 'height': 250}

graph_spinorama(g_spin, g_params) & \
    graph_freq(delta, g_delta) & \
    graph_spinorama(spin_filtered, g_params)

alt.VConcatChart(...)

In [48]:
print(spin_filtered, g_spin)

            Freq Measurements         dB
199      21.2402      On Axis -46.493349
200      21.9727      On Axis -44.737502
201      22.7051      On Axis -37.205957
202      23.4375      On Axis -39.855914
203      24.1699      On Axis -39.010873
...          ...          ...        ...
1587  17753.2000    DI offset   0.000000
1588  18379.4000    DI offset   0.000000
1589  19027.6000    DI offset   0.000000
1590  19698.5000    DI offset   0.000000
1591  19999.5000    DI offset   0.000000

[1393 rows x 3 columns]             Freq Measurements            dB
200      20.5078      On Axis -4.649262e+01
201      21.2402      On Axis -4.473672e+01
202      21.9727      On Axis -3.720512e+01
203      22.7051      On Axis -3.985502e+01
204      23.4375      On Axis -3.900992e+01
...          ...          ...           ...
1595  17753.2000    DI offset  2.131628e-14
1596  18379.4000    DI offset  2.131628e-14
1597  19027.6000    DI offset  2.131628e-14
1598  19698.5000    DI offset  2.131628e-14

In [63]:
df = pd.DataFrame({'f': [1,2,3]})
df1 = pd.DataFrame({'q1': [0,4,5,6]})
df2 = pd.DataFrame({'q2': [7,8,9]})

pd.concat([df, df1, df2], axis=1)

,f,q1,q2
0,1.0,0,7.0
1,2.0,4,8.0
2,3.0,5,9.0
3,NaN,6,NaN
